In [1]:
import sys
sys.path.append("../..")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import tensorflow as tf
from keras import layers
from datetime import datetime, timedelta
from sklearn.preprocessing import StandardScaler
import pickle
import config

In [2]:
name_model = 'RNN'

seq_length = 6 # model memory

train_csv_path = '../../data/dataframes/dfTrain.csv'
test_csv_path = '../../data/dataframes/dfTest.csv'
predictions_csv_path = './dfPredictions.csv'

model_pickle_path = './model.pickle' 
scaler_pickle_path = './scaler.pickle' 

In [3]:
df = pd.read_csv(train_csv_path)
df['time'] = pd.to_datetime(df['last_updated_dt'])

In [4]:
def df_to_X_y(data):
    x = []
    y = []

    for i in range(len(data)-seq_length):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length:i+seq_length+1]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [5]:
sc = StandardScaler()
training_data = sc.fit_transform(df['num_bikes_available'].values.reshape(-1, 1))

trainX, trainY = df_to_X_y(training_data)

In [6]:
#Model Creation
model = keras.Sequential()
model.add(layers.LSTM(32))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1))
model.build(input_shape=trainX.shape)
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (14393, 32)               4352      
                                                                 
 dropout (Dropout)           (14393, 32)               0         
                                                                 
 dense (Dense)               (14393, 1)                33        
                                                                 
Total params: 4385 (17.13 KB)
Trainable params: 4385 (17.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
#Complile
model.compile(
    loss=keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
    metrics=["mse"],
)

In [8]:
#Fit
batch_size = 4
history = model.fit(
    trainX, trainY, batch_size=batch_size, epochs=10
)

Epoch 1/10


3599/3599 [==============================] - 19s 4ms/step - loss: 0.1450 - mse: 0.1450
Epoch 2/10
3599/3599 [==============================] - 19s 5ms/step - loss: 0.1052 - mse: 0.1052
Epoch 3/10
3599/3599 [==============================] - 19s 5ms/step - loss: 0.1014 - mse: 0.1014
Epoch 4/10
3599/3599 [==============================] - 23s 6ms/step - loss: 0.0990 - mse: 0.0990
Epoch 5/10
3599/3599 [==============================] - 14s 4ms/step - loss: 0.0969 - mse: 0.0969
Epoch 6/10
3599/3599 [==============================] - 16s 4ms/step - loss: 0.0947 - mse: 0.0947
Epoch 7/10
3599/3599 [==============================] - 15s 4ms/step - loss: 0.0957 - mse: 0.0957
Epoch 8/10
3599/3599 [==============================] - 15s 4ms/step - loss: 0.0954 - mse: 0.0954
Epoch 9/10
3599/3599 [==============================] - 13s 4ms/step - loss: 0.0955 - mse: 0.0955
Epoch 10/10
3599/3599 [==============================] - 13s 4ms/step - loss: 0.0955 - mse: 0.0955


In [9]:
# Save
with open(model_pickle_path, 'wb') as file:
    pickle.dump(model, file)

with open(scaler_pickle_path, 'wb') as file:
    pickle.dump(sc, file)